# Coronavirus Data Analysis

In [1]:
import pandas as pd
import numpy as np

In [2]:
spark.sparkContext.applicationId

'application_1636170550178_0025'

In [3]:
pd.options.display.max_columns = 35

### Create Variables for Queries

In [4]:
# Look back days variables
daysInterval = 365
print(daysInterval)
maxDays = 365
print(maxDays)
oneYearInterval=365
print(oneYearInterval)

# Username variable
user_id = 'rc047407'           ####<-------------------- change to your user id (Ex: tl068507)
userName = 'user_' + user_id
print(userName)

# Algorithm name variable
algorithmName='CoronavirusLengthOfStay_HighLevel'
print(algorithmName)

365
365
365
user_rc047407
CoronavirusLengthOfStay_HighLevel


In [5]:
stdVar = spark.sql(f"""
  select  round(std(h.length_of_stay),2) as std_length_of_stay
       
  from {userName}.{algorithmName}_Hospital h
  
  left join {userName}.{algorithmName}_Vaccine v
    on h.personid = v.personId
      and h.tenant = v.tenant

   where    
      datediff(h.serviceDate, h.dischargeDate) < 0
      and h.length_of_stay > 0
       
""")
stdValue = stdVar.first()['std_length_of_stay']
print(stdValue)

12.8


In [6]:
stdVar = spark.sql(f"""
  select  round(std(h.length_of_stay),2) as std_length_of_stay
       
  from {userName}.{algorithmName}_Hospital h
  
  left join {userName}.{algorithmName}_Vaccine v
    on h.personid = v.personId
      and h.tenant = v.tenant

   where    
      datediff(h.serviceDate, h.dischargeDate) < 0
      and h.length_of_stay > 0
 
      and  ( datediff(h.serviceDate, v.vaccine_date) <= 0
             or v.vaccine_date is null) 
       
""")
stdValue_nonVac = stdVar.first()['std_length_of_stay']
print(stdValue_nonVac)

12.81


In [7]:
stdVar = spark.sql(f"""
  select  round(std(h.length_of_stay),2) as std_length_of_stay
       
  from {userName}.{algorithmName}_Hospital h
  
  left join {userName}.{algorithmName}_Vaccine v
    on h.personid = v.personId
      and h.tenant = v.tenant

   where    
      datediff(h.serviceDate, h.dischargeDate) < 0
      and h.length_of_stay > 0
               
      and  ( datediff(h.serviceDate, v.vaccine_date) > 0
             and v.vaccine_date is not null)        
      
    
""")
stdValue_vac = stdVar.first()['std_length_of_stay']
print(stdValue_vac)

5.62


In [56]:
spark.sql(f"""
  select  discharge_to
          , count(distinct personId) as count_of_non_vaccinated_patients
          , round(mean(length_of_stay),2) as mean_length_of_stay
   from (       
     select personId
            , max(discharge_to) as discharge_to
            , mean(length_of_stay) as length_of_stay
     from (       
          select
                  h.personId
                , case
                    when discharged_to_skilled_nursing_facility = 1 then "Skilled Nursing Facility"
                    when discharged_to_hospice = 1 then 'Hospice Care'
                    when died_in_the_hospital = 1 then 'Mortuary'
                    when discharged_to_rehab_facility = 1 then 'Rehab Facility'
                    when discharged_to_long_term_care_facility = 1 then 'Long Term Care Facility'
                    when discharged_to_skilled_nursing_facility = 0
                     and discharged_to_hospice = 0
                     and died_in_the_hospital = 0
                     and discharged_to_rehab_facility = 0
                     and discharged_to_long_term_care_facility = 0
                       then 'Home'
                   end as discharge_to
                , length_of_stay
             
                     
        from {userName}.{algorithmName}_Hospital h

         where    
              datediff(h.serviceDate, h.dischargeDate) < 0
              and h.length_of_stay > 0
              and h.length_of_stay < 2*{stdValue_vac}

              and  ( 
                    not exists (select 1
                                     from {userName}.{algorithmName}_Vaccine v2
                                     where  h.personid = v2.personId
                                      and h.tenant = v2.tenant
                                      and datediff(h.serviceDate, v2.vaccine_date) <= 0
                                    )  
                    )                
                    
              and deceased = 1      
        ) subquery 
          group by 1 
          
    ) subquery2 
     group by 1
     order by 1 
     
""").toPandas()

,discharge_to,count_of_non_vaccinated_patients,mean_length_of_stay
0,Home,2279,5.45
1,Hospice Care,737,6.45
2,Long Term Care Facility,45,6.77
3,Mortuary,11089,6.28
4,Rehab Facility,51,6.78
5,Skilled Nursing Facility,986,6.56


In [57]:
spark.sql(f"""
  select   count(distinct personId) as count_of_non_vaccinated_patients
          , round(mean(length_of_stay),2) as mean_length_of_stay
   from (       
     select personId
            , max(discharge_to) as discharge_to
            , mean(length_of_stay) as length_of_stay
     from (       
          select
                  h.personId
                , case
                    when discharged_to_skilled_nursing_facility = 1 then "Skilled Nursing Facility"
                    when discharged_to_hospice = 1 then 'Hospice Care'
                    when died_in_the_hospital = 1 then 'Mortuary'
                    when discharged_to_rehab_facility = 1 then 'Rehab Facility'
                    when discharged_to_long_term_care_facility = 1 then 'Long Term Care Facility'
                    when discharged_to_skilled_nursing_facility = 0
                     and discharged_to_hospice = 0
                     and died_in_the_hospital = 0
                     and discharged_to_rehab_facility = 0
                     and discharged_to_long_term_care_facility = 0
                       then 'Home'
                   end as discharge_to
                , length_of_stay
             
                     
        from {userName}.{algorithmName}_Hospital h
          
        left join {userName}.{algorithmName}_Vaccine v
            on h.personid = v.personId
              and h.tenant = v.tenant

           where    
              datediff(h.serviceDate, h.dischargeDate) < 0
              and h.length_of_stay > 0
              and h.length_of_stay < 2*{stdValue_vac}

              and  ( datediff(h.serviceDate, v.vaccine_date) <= 0
                     or not exists (select 1
                                     from {userName}.{algorithmName}_Vaccine v2
                                     where  h.personid = v2.personId
                                      and h.tenant = v2.tenant
                                    )  
                    )                
                    
              and deceased = 1      
        ) subquery 
          group by 1 
          
    ) subquery2    
     order by 1 
     
""").toPandas()

,count_of_non_vaccinated_patients,mean_length_of_stay
0,15182,6.19


In [50]:
spark.sql(f"""
  select  discharge_to 
          , count(distinct personId) as count_of_vaccinated_patients
          , round(mean(length_of_stay),2) as mean_length_of_stay
   from (       
     select personId
            , first(discharge_to) as discharge_to
            , mean(length_of_stay) as length_of_stay
     from (       
          select
                  h.personId
                , case
                    when discharged_to_skilled_nursing_facility = 1 then "Skilled Nursing Facility"
                    when discharged_to_hospice = 1 then 'Hospice Care'
                    when died_in_the_hospital = 1 then 'Mortuary'
                    when discharged_to_rehab_facility = 1 then 'Rehab Facility'
                    when discharged_to_long_term_care_facility = 1 then 'Long Term Care Facility'
                    when discharged_to_skilled_nursing_facility = 0
                     and discharged_to_hospice = 0
                     and died_in_the_hospital = 0
                     and discharged_to_rehab_facility = 0
                     and discharged_to_long_term_care_facility = 0
                       then 'Home'
                   end as discharge_to
                , length_of_stay
             
                     
        from {userName}.{algorithmName}_Hospital h
          
        left join {userName}.{algorithmName}_Vaccine v
            on h.personid = v.personId
              and h.tenant = v.tenant

           where    
              datediff(h.serviceDate, h.dischargeDate) < 0
              and h.length_of_stay > 0
              and h.length_of_stay < 2*{stdValue_vac}

             and  ( datediff(h.serviceDate, v.vaccine_date) >= 0
                     and exists (select 1
                                     from {userName}.{algorithmName}_Vaccine v2
                                     where  h.personid = v2.personId
                                      and h.tenant = v2.tenant
                                    )  
                   )                 
                    
              and deceased = 1      
        ) subquery 
          group by 1 
          
    ) subquery2    
     group by 1
     order by 1 
     
""").toPandas()

,discharge_to,count_of_vaccinated_patients,mean_length_of_stay
0,Home,1,3.00
1,Hospice Care,2,5.50
2,Mortuary,8,4.94


In [58]:
spark.sql(f"""
  select   count(distinct personId) as count_of_vaccinated_patients
          , round(mean(length_of_stay),2) as mean_length_of_stay
   from (       
     select personId
            , max(discharge_to) as discharge_to
            , mean(length_of_stay) as length_of_stay
     from (       
          select
                  h.personId
                , case
                    when discharged_to_skilled_nursing_facility = 1 then "Skilled Nursing Facility"
                    when discharged_to_hospice = 1 then 'Hospice Care'
                    when died_in_the_hospital = 1 then 'Mortuary'
                    when discharged_to_rehab_facility = 1 then 'Rehab Facility'
                    when discharged_to_long_term_care_facility = 1 then 'Long Term Care Facility'
                    when discharged_to_skilled_nursing_facility = 0
                     and discharged_to_hospice = 0
                     and died_in_the_hospital = 0
                     and discharged_to_rehab_facility = 0
                     and discharged_to_long_term_care_facility = 0
                       then 'Home'
                   end as discharge_to
                , length_of_stay
             
                     
        from {userName}.{algorithmName}_Hospital h
          
        left join {userName}.{algorithmName}_Vaccine v
            on h.personid = v.personId
              and h.tenant = v.tenant

           where    
              datediff(h.serviceDate, h.dischargeDate) < 0
              and h.length_of_stay > 0
              and h.length_of_stay < 2*{stdValue_vac}

              and  ( datediff(h.serviceDate, v.vaccine_date) > 0
                    and v.vaccine_date is not null)  
                    
              and deceased = 1      
        ) subquery 
          group by 1 
          
    ) subquery2    
     order by 1 
     
""").toPandas()

,count_of_vaccinated_patients,mean_length_of_stay
0,11,4.86


In [59]:
spark.sql(f"""
  select  discharge_to
          , count(distinct personId) as count_of_non_vaccinated_patients
          , round(mean(length_of_stay),2) as mean_length_of_stay
   from (       
     select personId
            , max(discharge_to) as discharge_to
            , mean(length_of_stay) as length_of_stay
     from (       
          select
                  h.personId
                , case
                    when discharged_to_skilled_nursing_facility = 1 then "Skilled Nursing Facility"
                    when discharged_to_hospice = 1 then 'Hospice Care'
                    when died_in_the_hospital = 1 then 'Mortuary'
                    when discharged_to_rehab_facility = 1 then 'Rehab Facility'
                    when discharged_to_long_term_care_facility = 1 then 'Long Term Care Facility'
                    when discharged_to_skilled_nursing_facility = 0
                     and discharged_to_hospice = 0
                     and died_in_the_hospital = 0
                     and discharged_to_rehab_facility = 0
                     and discharged_to_long_term_care_facility = 0
                       then 'Home'
                   end as discharge_to
                , length_of_stay
             
                     
        from {userName}.{algorithmName}_Hospital h

         where    
              datediff(h.serviceDate, h.dischargeDate) < 0
              and h.length_of_stay > 0
              and h.length_of_stay < 2*{stdValue_vac}

              and deceased = 1      
        ) subquery 
          group by 1 
          
    ) subquery2 
     group by 1
     order by 1 
     
""").toPandas()

,discharge_to,count_of_non_vaccinated_patients,mean_length_of_stay
0,Home,2283,5.45
1,Hospice Care,738,6.45
2,Long Term Care Facility,45,6.77
3,Mortuary,11089,6.28
4,Rehab Facility,51,6.78
5,Skilled Nursing Facility,986,6.56


In [61]:
spark.sql(f"""
    select count(distinct personId) as count_of_non_vaccinated_patients
          , round(mean(length_of_stay),2) as mean_length_of_stay
   from (       
     select personId
            , max(discharge_to) as discharge_to
            , mean(length_of_stay) as length_of_stay
     from (       
          select
                  h.personId
                , case
                    when discharged_to_skilled_nursing_facility = 1 then "Skilled Nursing Facility"
                    when discharged_to_hospice = 1 then 'Hospice Care'
                    when died_in_the_hospital = 1 then 'Mortuary'
                    when discharged_to_rehab_facility = 1 then 'Rehab Facility'
                    when discharged_to_long_term_care_facility = 1 then 'Long Term Care Facility'
                    when discharged_to_skilled_nursing_facility = 0
                     and discharged_to_hospice = 0
                     and died_in_the_hospital = 0
                     and discharged_to_rehab_facility = 0
                     and discharged_to_long_term_care_facility = 0
                       then 'Home'
                   end as discharge_to
                , length_of_stay
             
                     
        from {userName}.{algorithmName}_Hospital h

         where    
              datediff(h.serviceDate, h.dischargeDate) < 0
              and h.length_of_stay > 0
              and h.length_of_stay < 2*{stdValue_vac}

              and deceased = 1      
        ) subquery 
          group by 1 
          
    ) subquery2 
     order by 1 
     
""").toPandas()

,count_of_non_vaccinated_patients,mean_length_of_stay
0,15192,6.19
